#LOAD DATA

In [15]:
import pandas as pd


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
 ! pip install -q kaggle

In [18]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/CAPSTONE/Husein/Collaborative_filterring_learn"

In [19]:
%cd /content/drive/MyDrive/CAPSTONE/Husein/Collaborative_filterring_learn


/content/drive/MyDrive/CAPSTONE/Husein/Collaborative_filterring_learn


In [20]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

the-movies-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [21]:
!ls -a

Collaborative_filterring_learn.ipynb  movies_metadata.csv
credits.csv			      ratings.csv
kaggle.json			      ratings_small.csv
keywords.csv			      Simpler_Recomender.ipynb
links.csv			      the-movies-dataset.zip
links_small.csv


#BUILD

Simple Recommender System based on the metric below:

weightedRating(WR) = ((v/v+m).R) + ((m/v+m).C)

v --> is the number of votes for the movie. (vote_count)

m --> is the min votes required to be listed in chart. (based on negative vote)

R --> is the average rating of the movie. (vote_average)

C --> is the mean vote across the the whole report. (calculate from data)


In [22]:
import pandas as pd

In [23]:
metadata=pd.read_csv('/content/drive/MyDrive/CAPSTONE/Husein/Collaborative_filterring_learn/movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [25]:
print(metadata.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [26]:
C=metadata['vote_average'].mean()
print(C)

5.618207215133889


In [33]:
m=metadata['vote_count'].quantile(0.90)
m

160.0

In [34]:
q_movies=metadata.copy().loc[metadata['vote_count']>=m]
q_movies.shape

(4555, 24)

In [29]:
q_movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [30]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [36]:
#Buat column baru dengan nama score biar tau urutan rating film terbaik
q_movies['score']=q_movies.apply(weighted_rating,axis=1)

In [41]:
#Panggil data yang sudah terurut :
q_movies.sort_values(by=['score'],ascending=False,inplace=True)
q_movies[['title','vote_count','score']].head()

,title,vote_count,score
314,The Shawshank Redemption,8358.0,8.445869
834,The Godfather,6024.0,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,8.421453
12481,The Dark Knight,12269.0,8.265477
2843,Fight Club,9678.0,8.256385
